In [316]:
import pandas as pd
import datetime
import numpy as np
import sklearn
from sklearn.cross_validation import train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import *
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
weekend = ['Saturday','Sunday']
def week_day_type(x):
    if x in weekend:
        return 'weekends'
    else:
        return 'weekdays'
def time_slot(x):
    if x in morning:
        return 'morning'
    elif x in afternoon:
        return 'afternoon'
    elif x in evening:
        return 'evening'
    else:
        return 'night'  
df=pd.read_csv("C:/Users/Akash/Documents/ADS/Assignment2/Appliances-energy-prediction-data-master/energydata_complete.csv")
df['date']=pd.to_datetime(df['date'])
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df['day_of_week']=df['date'].dt.weekday_name
df['time_hr_24']=df['date'].dt.hour
df['time_min']=df['date'].dt.minute
df['week_day_type']=df['day_of_week'].map(week_day_type)
morning=range(6,12)
afternoon=range(12,17)
evening=range(17,22)  
df['time_slot']=df['time_hr_24'].map(time_slot)
df.drop(['date'],axis=1,inplace=True)
df=pd.get_dummies(df,prefix=['DOW','TS','WDT'],columns=['day_of_week','time_slot','week_day_type'])
#x_train.isnull().sum()
#df.dtypes

In [317]:
print(df.shape)
df=df[(df['Appliances']-df['Appliances'].mean()).abs() <= 3*df['Appliances'].std()]
df=df[(df['lights']-df['lights'].mean()).abs() <= 3*df['lights'].std()]
df=df[(df['T1']-df['T1'].mean()).abs() <= 3*df['T1'].std()]
df=df[(df['T2']-df['T2'].mean()).abs() <= 3*df['T2'].std()]
df=df[(df['T3']-df['T3'].mean()).abs() <= 3*df['T3'].std()]
df=df[(df['T4']-df['T4'].mean()).abs() <= 3*df['T4'].std()]
df=df[(df['T5']-df['T5'].mean()).abs() <= 3*df['T5'].std()]
df=df[(df['T6']-df['T6'].mean()).abs() <= 3*df['T6'].std()]
df=df[(df['T7']-df['T7'].mean()).abs() <= 3*df['T7'].std()]
df=df[(df['T8']-df['T8'].mean()).abs() <= 3*df['T8'].std()]
df=df[(df['T9']-df['T9'].mean()).abs() <= 3*df['T9'].std()]
df=df[(df['T_out']-df['T_out'].mean()).abs() <= 3*df['T_out'].std()]
df=df[(df['RH_1']-df['RH_1'].mean()).abs() <= 3*df['RH_1'].std()]
df=df[(df['RH_2']-df['RH_2'].mean()).abs() <= 3*df['RH_2'].std()]
df=df[(df['RH_3']-df['RH_3'].mean()).abs() <= 3*df['RH_3'].std()]
df=df[(df['RH_4']-df['RH_4'].mean()).abs() <= 3*df['RH_4'].std()]
df=df[(df['RH_5']-df['RH_5'].mean()).abs() <= 3*df['RH_5'].std()]
df=df[(df['RH_6']-df['RH_6'].mean()).abs() <= 3*df['RH_6'].std()]
df=df[(df['RH_7']-df['RH_7'].mean()).abs() <= 3*df['RH_7'].std()]
df=df[(df['RH_8']-df['RH_8'].mean()).abs() <= 3*df['RH_8'].std()]
df=df[(df['RH_9']-df['RH_9'].mean()).abs() <= 3*df['RH_9'].std()]
df=df[(df['RH_out']-df['RH_out'].mean()).abs() <= 3*df['RH_out'].std()]
df=df[(df['Press_mm_hg']-df['Press_mm_hg'].mean()).abs() <= 3*df['Press_mm_hg'].std()]
df=df[(df['Windspeed']-df['Windspeed'].mean()).abs() <= 3*df['Windspeed'].std()]
df=df[(df['Visibility']-df['Visibility'].mean()).abs() <= 3*df['Visibility'].std()]
df=df[(df['Tdewpoint']-df['Tdewpoint'].mean()).abs() <= 3*df['Tdewpoint'].std()]
df=df[(df['rv1']-df['rv1'].mean()).abs() <= 3*df['rv1'].std()]
df=df[(df['rv2']-df['rv2'].mean()).abs() <= 3*df['rv2'].std()]
print(df.shape)

(19735, 46)
(17118, 46)


In [331]:
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
print(df_train.shape)
print(df_test.shape)
x_train=df_train.iloc[:,1:]
y_train=df_train['Appliances']
drop_col_list=['year','DOW_Friday','DOW_Monday','DOW_Saturday','DOW_Sunday','DOW_Thursday','DOW_Tuesday','DOW_Wednesday','TS_afternoon',
              'WDT_weekdays','WDT_weekends'
               ,'TS_morning','month','time_min','TS_evening','day','rv1','rv2','Visibility','Windspeed','T9','lights']
#               ,'DOW_Friday','TS_evening','day','rv1','rv2','Visibility','TS_morning',
#               ,'T9','T7','lights']
x_train.drop(drop_col_list,axis=1,inplace=True)
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)

(11982, 46)
(5136, 46)


In [332]:
rf=RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=25,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
#'n_estimators': 822, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True
rf.fit(x_train_sc, y_train)
y_train_pred=rf.predict(x_train_sc)
print("R2   :",r2_score(y_train,y_train_pred))
print("MAE  :",mean_absolute_error(y_train,y_train_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_train,y_train_pred)))
print("MAPE :",mean_absolute_percentage_error(y_train,y_train_pred))

R2   : 0.999209775874
MAE  : 1.08126035694
RMSE : 1.88259496885
MAPE : 1.66006504566


In [333]:
feature_list = list(x_train.columns)
importances = list(rf.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: time_hr_24           Importance: 0.08
Variable: RH_3                 Importance: 0.06
Variable: TS_night             Importance: 0.06
Variable: RH_8                 Importance: 0.05
Variable: Press_mm_hg          Importance: 0.05
Variable: T1                   Importance: 0.04
Variable: RH_1                 Importance: 0.04
Variable: T2                   Importance: 0.04
Variable: RH_2                 Importance: 0.04
Variable: T3                   Importance: 0.04
Variable: RH_4                 Importance: 0.04
Variable: RH_5                 Importance: 0.04
Variable: T6                   Importance: 0.04
Variable: RH_6                 Importance: 0.04
Variable: RH_7                 Importance: 0.04
Variable: T8                   Importance: 0.04
Variable: RH_9                 Importance: 0.04
Variable: T_out                Importance: 0.04
Variable: RH_out               Importance: 0.04
Variable: Tdewpoint            Importance: 0.04
Variable: T4                   Importanc

In [334]:
x_test=df_test.iloc[:,1:]
x_test.drop(drop_col_list,axis=1,inplace=True)
#x_test=x_test[col_list]
y_test=df_test['Appliances']
x_test_sc=scaler.transform(x_test)
y_test_pred=rf.predict(x_test_sc)
print("R2   :",r2_score(y_test,y_test_pred))
print("MAE  :",mean_absolute_error(y_test,y_test_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_test_pred)))
print("MAPE :",mean_absolute_percentage_error(y_test,y_test_pred))

R2   : 0.567140276074
MAE  : 20.5078278131
RMSE : 42.4546707049
MAPE : 24.765400843


In [ ]:
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
print(df_train.shape)
print(df_test.shape)
x_train=df_train.iloc[:,1:]
y_train=df_train['Appliances']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,1:]
y_test=df_test['Appliances']
x_test_sc=scaler.transform(x_test)
#Random Hyperparameter Grid
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
rdf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rdf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train_sc, y_train)
print(rf_random.best_params_)
model=rf_random.best_estimator_
#evaluate(model,x_train,y_train)
y_train_pred=model.predict(x_train_sc)
print("R2   :",r2_score(y_train,y_train_pred))
print("MAE  :",mean_absolute_error(y_train,y_train_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_train,y_train_pred)))
print("MAPE :",mean_absolute_percentage_error(y_train,y_train_pred))

In [293]:
# col_list=['time_hr_24','RH_1','RH_2','T3','RH_3','Press_mm_hg','TS_night','T1','T2','T4','RH_4','T5','RH_5','T6','RH_6',
#                         'T7','RH_7','T8','T9','RH_8','RH_9','T_out','RH_out','Windspeed','Tdewpoint','lights']
# col_list=['time_hr_24','RH_1','RH_2','T3','RH_3','Press_mm_hg','TS_night','T2','RH_4','RH_5','T6','RH_6',
#                          'RH_7','T8','RH_8','RH_9','T_out','RH_out','Tdewpoint']
drop_col_list=['year','DOW_Monday','DOW_Saturday','DOW_Sunday','DOW_Thursday','DOW_Tuesday','DOW_Wednesday','TS_afternoon','TS_morning',
              'WDT_weekdays','WDT_weekends','month','time_min','DOW_Friday','TS_evening','day','rv1','rv2','Visibility'
               ,'T9','T7','lights'
#               ,'RH_out','T8','T5'
#               ,'T1','T4','T5'
              ]
x_train.drop(drop_col_list,axis=1,inplace=True)
# x_train=x_train[col_list]
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)

In [294]:
#rf=RandomForestRegressor(n_estimators=822,max_features='sqrt',random_state=42)
rf=RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=25,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
#'n_estimators': 822, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True
rf.fit(x_train_sc, y_train)
y_train_pred=rf.predict(x_train_sc)
print("R2   :",r2_score(y_train,y_train_pred))
print("MAE  :",mean_absolute_error(y_train,y_train_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_train,y_train_pred)))
print("MAPE :",mean_absolute_percentage_error(y_train,y_train_pred))

R2   : 0.999320682589
MAE  : 1.40899175619
RMSE : 2.68343110405
MAPE : 1.85289375652


In [295]:
feature_list = list(x_train.columns)
importances = list(rf.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: time_hr_24           Importance: 0.07
Variable: RH_2                 Importance: 0.05
Variable: T3                   Importance: 0.05
Variable: RH_3                 Importance: 0.05
Variable: RH_5                 Importance: 0.05
Variable: RH_8                 Importance: 0.05
Variable: Press_mm_hg          Importance: 0.05
Variable: T1                   Importance: 0.04
Variable: RH_1                 Importance: 0.04
Variable: T2                   Importance: 0.04
Variable: T4                   Importance: 0.04
Variable: RH_4                 Importance: 0.04
Variable: T5                   Importance: 0.04
Variable: T6                   Importance: 0.04
Variable: RH_6                 Importance: 0.04
Variable: RH_7                 Importance: 0.04
Variable: T8                   Importance: 0.04
Variable: RH_9                 Importance: 0.04
Variable: T_out                Importance: 0.04
Variable: RH_out               Importance: 0.04
Variable: Tdewpoint            Importanc

In [296]:
x_test=df_test.iloc[:,1:]
x_test.drop(drop_col_list,axis=1,inplace=True)
#x_test=x_test[col_list]
y_test=df_test['Appliances']
x_test_sc=scaler.transform(x_test)
y_test_pred=rf.predict(x_test_sc)
print("R2   :",r2_score(y_test,y_test_pred))
print("MAE  :",mean_absolute_error(y_test,y_test_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_test_pred)))
print("MAPE :",mean_absolute_percentage_error(y_test,y_test_pred))

R2   : 0.630026410669
MAE  : 28.2517975553
RMSE : 61.7389497672
MAPE : 27.1929001659


In [218]:
#Random Hyperparameter Grid
# Number of trees in random forest
n_estimators = [288] #[int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt'] #['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [5,10,15,20,25,30,35,40,45,50] #[int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2] # [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rdf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rdf, param_distributions = random_grid, n_iter = 66, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train_sc, y_train)
print(rf_random.best_params_)
model=rf_random.best_estimator_
#evaluate(model,x_train,y_train)
y_train_pred=model.predict(x_train_sc)
print("R2   :",r2_score(y_train,y_train_pred))
print("MAE  :",mean_absolute_error(y_train,y_train_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_train,y_train_pred)))
print("MAPE :",mean_absolute_percentage_error(y_train,y_train_pred))

{'n_estimators': [288], 'max_features': ['sqrt'], 'max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, None], 'min_samples_split': [2], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
Fitting 3 folds for each of 66 candidates, totalling 198 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 198 out of 198 | elapsed:  7.4min finished


{'n_estimators': 288, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 25, 'bootstrap': False}
R2   : 0.999219122303
MAE  : 1.48404136001
RMSE : 2.87703792091
MAPE : 1.93288968935


In [219]:
feature_list = list(x_train.columns)
importances = list(model.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: time_hr_24           Importance: 0.07
Variable: RH_1                 Importance: 0.05
Variable: RH_2                 Importance: 0.05
Variable: T3                   Importance: 0.05
Variable: RH_3                 Importance: 0.05
Variable: RH_5                 Importance: 0.05
Variable: RH_8                 Importance: 0.05
Variable: Press_mm_hg          Importance: 0.05
Variable: TS_night             Importance: 0.05
Variable: T1                   Importance: 0.04
Variable: T2                   Importance: 0.04
Variable: T4                   Importance: 0.04
Variable: RH_4                 Importance: 0.04
Variable: T5                   Importance: 0.04
Variable: T6                   Importance: 0.04
Variable: RH_6                 Importance: 0.04
Variable: RH_7                 Importance: 0.04
Variable: T8                   Importance: 0.04
Variable: RH_9                 Importance: 0.04
Variable: T_out                Importance: 0.04
Variable: RH_out               Importanc

In [220]:
rf_random.best_estimator_ #new

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=25,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=288, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [5]:
# RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
#            max_features='sqrt', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=288, n_jobs=1,
#            oob_score=False, random_state=None, verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=288, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [221]:
x_test=df_test.iloc[:,1:]
x_test.drop(drop_col_list,axis=1,inplace=True)
#x_test=x_test[col_list]
y_test=df_test['Appliances']
x_test_sc=scaler.transform(x_test)
y_test_pred=model.predict(x_test_sc)
print("R2   :",r2_score(y_test,y_test_pred))
print("MAE  :",mean_absolute_error(y_test,y_test_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_test,y_test_pred)))
print("MAPE :",mean_absolute_percentage_error(y_test,y_test_pred))

R2   : 0.626357854918
MAE  : 28.3932633018
RMSE : 62.0442878945
MAPE : 27.3316675501


In [278]:
y_test_pred.dtype

dtype('float64')

Linear Regression

In [ ]:
lm=linear_model.LinearRegression()
mod=lm.fit(x_train,y_train)
y_train_pred=lm.predict(x_train)
print("R2   :",r2_score(y_train,y_train_pred))
print("MAE  :",mean_absolute_error(y_train,y_train_pred))
print("RMSE :",np.sqrt(mean_squared_error(y_train,y_train_pred)))
print("MAPE :",mean_absolute_percentage_error(y_train,y_train_pred))